# relation_df Dataset

Creación de dataset relation_df que buscar relacionar los dataset de google maps y yelp mediante las columnas 'latitude' y 'longitude'

In [427]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [428]:
from pathlib import Path

# Definir la ruta relativa desde el directorio del script
relative_path_business = Path("datasets") / "business.parquet"
relative_path_pizza = Path("datasets") / "pizza_metadata.parquet"

# Cargar el archivo
business = pd.read_parquet(relative_path_business)
pizza_metadata = pd.read_parquet(relative_path_pizza)


In [429]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,None,93101,34.426679,-119.711197,5.0,7
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,None,63123,38.551126,-90.335695,3.0,15
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,None,85711,32.223236,-110.880452,3.5,22
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13


In [430]:
pizza_metadata.head()

,name,address,gmap_id,latitude,longitude,avg_rating,num_of_reviews
0,Primo Pizza & Subs,"Primo Pizza & Subs, 148 W St Joseph St, Easton...",0x89c46d5e4554eae1:0xa2f8b211524ca29a,40.682177,-75.209820,4.5,25
1,Sir Pizza Xpress,"Sir Pizza Xpress, 201 E Broad St, Smithville, ...",0x8866cef69c5737a1:0xbdf838584dca0b9c,35.956944,-85.811111,4.1,18
2,Bucci's Greek & Italian Specialties,"Bucci's Greek & Italian Specialties, 8030 S Ho...",0x876c8410e0d7ec6d:0xb06492857fa9ec9d,39.572697,-104.922399,4.7,13
3,Dynamite Pizza & BBQ,"Dynamite Pizza & BBQ, 59069 Gratiot Ave, New H...",0x882518a534093375:0x4333d5f58f7af4c,42.735038,-82.785340,4.1,8
4,Three Star Pizza,"Three Star Pizza, 409 Cabot St #1, Beverly, MA...",0x89e3169821e62d4d:0x14ff0683c1ebca0e,42.559072,-70.881542,3.9,48


Join mediante 'longitude' y 'latitude'

In [431]:
epsilon = 0.0001  # Margen de tolerancia en coordenadas

# Renombrar columnas para evitar conflictos en el merge
business = business.rename(columns={"latitude": "latitude_business", "longitude": "longitude_business"})
pizza_metadata = pizza_metadata.rename(columns={"latitude": "latitude_metadata", "longitude": "longitude_metadata"})

# Merge manteniendo solo los negocios en el dataset business
relation_df = business.merge(
    pizza_metadata,
    how="left"  # Asegura que solo se mantengan los datos de business
)

# Filtrar solo las coincidencias dentro del margen epsilon
relation_df = relation_df.query(
    "abs(latitude_business - latitude_metadata) <= @epsilon and abs(longitude_business - longitude_metadata) <= @epsilon"
)[["name", "business_id", "gmap_id", "latitude_business", "longitude_business"]]

# Renombrar las columnas finales
relation_df = relation_df.rename(columns={"latitude_business": "latitude", "longitude_business": "longitude"})

# Mostrar las primeras filas
relation_df.head()

,name,business_id,gmap_id,latitude,longitude


No hay coincidencias usando 'latitude' y 'longitude'

In [432]:
print(business[["latitude_business", "longitude_business"]].head(5))
print(pizza_metadata[["latitude_metadata", "longitude_metadata"]].head(5))

   latitude_business  longitude_business
0          34.426679         -119.711197
1          38.551126          -90.335695
2          32.223236         -110.880452
3          39.955505          -75.155564
4          40.338183          -75.471659
   latitude_metadata  longitude_metadata
0          40.682177          -75.209820
1          35.956944          -85.811111
2          39.572697         -104.922399
3          42.735038          -82.785340
4          42.559072          -70.881542


'address' de pizza_metadata necesita normalización

In [433]:
print(business['address'].head(5))
print(pizza_metadata['address'].head(5))

0             1616 Chapala St, Ste 2
1    87 Grasso Plaza Shopping Center
2               5255 E Broadway Blvd
3                        935 Race St
4                      101 Walnut St
Name: address, dtype: object
0    Primo Pizza & Subs, 148 W St Joseph St, Easton...
1    Sir Pizza Xpress, 201 E Broad St, Smithville, ...
2    Bucci's Greek & Italian Specialties, 8030 S Ho...
3    Dynamite Pizza & BBQ, 59069 Gratiot Ave, New H...
4    Three Star Pizza, 409 Cabot St #1, Beverly, MA...
Name: address, dtype: object


In [434]:
# Buscar valores NaN en 'address'
pizza_metadata['address'].isna().sum()

108

In [435]:
print(pizza_metadata[pizza_metadata['address'].isna() == True]['name'].value_counts())

name
P.Za Kitchen                        4
Bite Squad                          2
Lorenzo's of New York               2
Rickenbacher Pizza                  1
Treehouse Deliveries                1
                                   ..
Da Pizza Dude                       1
Bella Rizzo's Pizzeria and Grill    1
Harmony Pizza-We Deliver            1
The Bike Waiter                     1
Closed                              1
Name: count, Length: 103, dtype: int64


In [436]:
# Eliminar filas con valores NaN en columna 'address' 
pizza_metadata = pizza_metadata.dropna(subset=['address'])

In [437]:
# Normalizar columna 'address' y se cambia nombre a 'old_address'
# Se extraer solo la parte de la dirección en pizza_metadata, eliminando el nombre del negocio
pizza_metadata["old_address"] = pizza_metadata["address"].str.split(",", n=1).str[1].str.strip()

print(pizza_metadata['old_address'].head(5))

0      148 W St Joseph St, Easton, PA 18042
1      201 E Broad St, Smithville, TN 37166
2     8030 S Holly St, Centennial, CO 80122
3    59069 Gratiot Ave, New Haven, MI 48048
4        409 Cabot St #1, Beverly, MA 01915
Name: old_address, dtype: object


Nota: 'address' de pizza_metadata contienen otra información, tal como la ciudad y el estado

El siguiente código separa los componentes de pizza_metadata['address'] en distintas columnas según corresponda

In [438]:
# Dividir la dirección en partes
split_address = pizza_metadata["old_address"].str.rsplit(",", n=2, expand=True)  # Divide desde el final

# Asignar cada parte a nuevas columnas
pizza_metadata["address"] = split_address[0].str.strip()
pizza_metadata["city"] = split_address[1].str.strip()

# Separar state y postal_code correctamente
state_zip_split = split_address[2].str.strip().str.split(" ", n=1, expand=True)
pizza_metadata["state"] = state_zip_split[0].str.strip()  # Estado (Ej: "GA")
pizza_metadata["postal_code"] = state_zip_split[1].str.strip()  # Código Postal (Ej: "31522")

# Mostrar los primeros registros para verificar
print(pizza_metadata[["old_address", "address", "city", "state", "postal_code"]].head(10))


                              old_address             address         city  \
0    148 W St Joseph St, Easton, PA 18042  148 W St Joseph St       Easton   
1    201 E Broad St, Smithville, TN 37166      201 E Broad St   Smithville   
2   8030 S Holly St, Centennial, CO 80122     8030 S Holly St   Centennial   
3  59069 Gratiot Ave, New Haven, MI 48048   59069 Gratiot Ave    New Haven   
4      409 Cabot St #1, Beverly, MA 01915     409 Cabot St #1      Beverly   
5    1104 1st St E, Park Rapids, MN 56470       1104 1st St E  Park Rapids   
6  2725 Manitowoc Rd, Green Bay, WI 54311   2725 Manitowoc Rd    Green Bay   
7   10637 N Tatum Blvd, Phoenix, AZ 85028  10637 N Tatum Blvd      Phoenix   
8  790 Washington Ave, Brooklyn, NY 11238  790 Washington Ave     Brooklyn   
9     3909 Creek Rd, Youngstown, NY 14174       3909 Creek Rd   Youngstown   

  state postal_code  
0    PA       18042  
1    TN       37166  
2    CO       80122  
3    MI       48048  
4    MA       01915  
5    MN  

In [439]:
# Revisar cuantos valores NaN hay en las columnas anteriores
for column in ["old_address", "address", "city", "state", "postal_code"]:
    
    print(f"{column}:", pizza_metadata[column].isna().sum())

old_address: 0
address: 0
city: 1
state: 105
postal_code: 105


In [440]:
# Eliminar filas con valores NaN en las columnas 'city' y 'state'
pizza_metadata = pizza_metadata.dropna(subset=['city'])
pizza_metadata = pizza_metadata.dropna(subset=['state'])

In [441]:
# Revisar número de filas de pizza_metadata
pizza_metadata.shape[0]

25175

In [442]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude_business', 'longitude_business', 'stars', 'review_count'],
      dtype='object')

In [443]:
business['address'].head()

0             1616 Chapala St, Ste 2
1    87 Grasso Plaza Shopping Center
2               5255 E Broadway Blvd
3                        935 Race St
4                      101 Walnut St
Name: address, dtype: object

In [444]:
pizza_metadata.columns

Index(['name', 'address', 'gmap_id', 'latitude_metadata', 'longitude_metadata',
       'avg_rating', 'num_of_reviews', 'old_address', 'city', 'state',
       'postal_code'],
      dtype='object')

In [445]:
pizza_metadata['address'].head()

0    148 W St Joseph St
1        201 E Broad St
2       8030 S Holly St
3     59069 Gratiot Ave
4       409 Cabot St #1
Name: address, dtype: object

El siguiente código trata de unir ambos datasets mediante la columna 'address'

In [446]:
# # Unir business con pizza_metadata usando "address" como clave
# merged_df = business.merge(
#     pizza_metadata,  # Dataset más grande
#     on=["address"],  # Unir por dirección
#     how="left",  # Mantiene todas las filas de "business" y solo une coincidencias en "pizza_metadata"
#     suffixes=("_business", "_metadata")  # Evita nombres duplicados en las columnas
# )

# # Seleccionar solo las columnas relevantes
# # merged_df = merged_df[[
# #     "business_id", "name_business", "address", "gmap_id", 
# #     "latitude_business", "longitude_business", "latitude_metadata", "longitude_metadata", 
# #     "stars", "review_count", "avg_rating", "num_of_reviews", "city", "state", "postal_code"
# # ]]

# # Mostrar las primeras filas
# merged_df.head()

In [447]:
# Renombrar la columna "address" en pizza_metadata antes de hacer el merge
pizza_metadata = pizza_metadata.rename(columns={"address": "address_google"})
business = business.rename(columns={"address" : "address_yelp"})

# Unir business con pizza_metadata usando "address" como clave
merged_df = business.merge(
    pizza_metadata,  # Dataset más grande
    left_on="address_yelp", 
    right_on="address_google",  # Usamos la dirección renombrada
    how="left",  # Mantiene todas las filas de "business"
    suffixes=("_yelp", "_google")  # Evita nombres duplicados
)


# Seleccionar solo las columnas deseadas, excluyendo latitude y longitude
merged_df = merged_df[[
    "business_id", "gmap_id", "name_yelp", "name_google", 
    "address_yelp", "address_google", "city_yelp", "city_google", 
    "state_yelp", "state_google", "postal_code_yelp", "postal_code_google"
]]

# Mostrar las primeras filas
merged_df.head()

,business_id,gmap_id,name_yelp,name_google,address_yelp,address_google,city_yelp,city_google,state_yelp,state_google,postal_code_yelp,postal_code_google
0,Pns2l4eNsfO8kk83dixA6A,NaN,"Abby Rappoport, LAC, CMQ",NaN,"1616 Chapala St, Ste 2",NaN,Santa Barbara,NaN,None,NaN,93101,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,NaN,The UPS Store,NaN,87 Grasso Plaza Shopping Center,NaN,Affton,NaN,None,NaN,63123,NaN
2,tUFrWirKiKi_TAnsVWINQQ,0x86d66fc7389c6efb:0xcb778e450955f8bf,Target,Pizza Hut Express,5255 E Broadway Blvd,5255 E Broadway Blvd,Tucson,Tucson,None,AZ,85711,85711
3,MTSW4McQd7CbVtyjqoe9mw,NaN,St Honore Pastries,NaN,935 Race St,NaN,Philadelphia,NaN,CA,NaN,19107,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,0x88547dee22d98613:0x4c151868224666ed,Perkiomen Valley Brewery,Steve's Pizza,101 Walnut St,101 Walnut St,Green Lane,Badin,MO,NC,18054,28009


In [448]:
merged_df.shape

(150825, 12)

In [449]:
relation_df = merged_df[merged_df['address_google'].isna() == False]

relation_df.shape

(3144, 12)

In [450]:
relation_df.head(10)

,business_id,gmap_id,name_yelp,name_google,address_yelp,address_google,city_yelp,city_google,state_yelp,state_google,postal_code_yelp,postal_code_google
2,tUFrWirKiKi_TAnsVWINQQ,0x86d66fc7389c6efb:0xcb778e450955f8bf,Target,Pizza Hut Express,5255 E Broadway Blvd,5255 E Broadway Blvd,Tucson,Tucson,None,AZ,85711,85711
4,mWMc6_wTdE0EUBKIGXDVfA,0x88547dee22d98613:0x4c151868224666ed,Perkiomen Valley Brewery,Steve's Pizza,101 Walnut St,101 Walnut St,Green Lane,Badin,MO,NC,18054,28009
31,Mjboz24M9NlBeiOJKLEd_Q,0x89c6b9c7c9fe6a33:0xbd9ddddd88aae695,DeSandro on Main,Pizzeria L’Angolo,4105 Main St,4105 Main St,Philadelphia,Philadelphia,PA,PA,19127,19127
88,LhpPSrulqVeTyJeK2xydvQ,0x8816d532acf5a675:0xdd4e611f1a57338e,Fresh Fruits & Salads,Third Street Pizzeria,114 N 3rd St,114 N 3rd St,Camden,Niles,CA,MI,08102,49120
147,uCQqtYW_xc1EJRm7NyakdA,0x80993f4534a09705:0x97a12e051ff8176b,Yogurt Beach,California Pizza Kitchen,2500 E 2nd St,2500 E 2nd St,Reno,Reno,IN,NV,89595,89595
256,py5aKmlTB2NarfsfcOpHOQ,0x88c2c238bae56661:0xa4684b03a202cf2d,Cafe Con Leche,Bavaro's Pizza Napoletana & Pastaria,4100 George J Bean Pkwy,4100 George J Bean Pkwy,Tampa,Tampa,LA,FL,33607,33607
292,rZw9O5lJ36m_mXeRKE4G9A,0x89c6f5f2fe557339:0x30b76bb0b1276717,La Sponda,Barra Rossa Ristorante,20 E Lancaster Ave,20 E Lancaster Ave,Downingtown,Downingtown,IN,PA,19335,19335
431,ifjluUv4VASwmFqEp8cWlQ,0x80e914e976e10c81:0x6eb5ef854c57c0b2,Marty's Pizza,Marty's Pizza,2733 De La Vina St,2733 De La Vina St,Santa Barbara,Santa Barbara,PA,CA,93105,93105
601,xwSWtJcQkzTF6HNm_IMgcg,0x80e940a57b4bc689:0x576cb9b73973dda0,Rusty's Pizza Parlor,Rusty's Pizza Parlor,270 Storke Rd,270 Storke Rd,Goleta,Goleta,FL,CA,93117,93117
652,kGjpBqAqG4mIuFSgU2KKEA,0x89c6c639a8ff8b87:0xdfbf6ec28977036c,The Royal Indian Cuisine,The Spot,272 S 20th St,272 S 20th St,Philadelphia,Philadelphia,TN,PA,19103,19103
